In [32]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random
from datetime import datetime, timedelta

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'plotnine'

## Load data

In [6]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [36]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 1.4 s, sys: 6.79 ms, total: 1.41 s
Wall time: 1.53 s


In [41]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

#iterrows returns a tuple so need the (i,r)

CPU times: user 863 ms, sys: 9.71 ms, total: 873 ms
Wall time: 875 ms


In [45]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 335 ms, sys: 6.79 ms, total: 342 ms
Wall time: 339 ms


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

In [44]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 3.2 ms, sys: 2.78 ms, total: 5.98 ms
Wall time: 6.9 ms


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

# Practice

In [18]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']] #subsetted data frame and then slicing to the columns of interest
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
941,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00
14013,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [7]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as shape
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 6048       DAY  OTHERS          THEFT/OTHER   
         11308      DAY  OTHERS         THEFT F/AUTO   
         13946      DAY  OTHERS          THEFT/OTHER   
         16387      DAY  OTHERS         THEFT F/AUTO   
         23620  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 6048        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         11308  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         13946    1724 - 1799 BLOCK OF 17TH STREET NW  396662.0  138429.0   
         16387       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         23620       1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 6048   ...        5801.0    Precinct 129  38.899121 -77.021926   
         11308  ...        4100.0     Precinct 13  38.913346 -77.050526   
         13946  ...        5302.0     Precinct 15  38.913720 -77.038489   
         16387  ...        5303.0     Precinct 15  38.909638 -77.040089   
         23620  ...       10700.0     Precinct 17  38.905648 -77.039822   

                            BID              START_DATE  \
20123250 6048          DOWNTOWN  2020/08/29 16:05:40+00   
         11308              NaN  2020/08/26 22:00:29+00   
         13946              NaN  2020/08/28 20:55:00+00   
         16387              NaN  2020/08/28 22:00:23+00   
         23620  GOLDEN TRIANGLE  2020/08/27 19:01:24+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 6048   2020/08/29 16:08:33+00  500137941             NaN   
         11308  2020/08/27 12:00:51+00  500147812             NaN   
         13946  2020/08/28 21:05:00+00  500157540             NaN   
         16387  2020/08/29 08:00:27+00  500183867             NaN   
         23620  2020/08/29 19:00:05+00  500366738             NaN   

                               report_dt  
20123250 6048  2020-08-29 17:15:19+00:00  
         11308 2020-08-29 16:45:57+00:00  
         13946 2020-08-29 14:29:59+00:00  
         16387 2020-08-29 16:05:18+00:00  
         23620 2020-08-29 22:04:46+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [8]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches) #helpful to put into one data frame
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 491   -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         10824 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         14887 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         16277 -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         16280 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 491     EVENING  OTHERS          THEFT/OTHER   
         10824       DAY  OTHERS          THEFT/OTHER   
         14887   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16277  MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         16280       DAY  OTHERS         THEFT F/AUTO   

                                                     BLOCK    XBLOCK  \
20165648 491    300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         10824   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         14887          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         16277            100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         16280              300 - 399 BLOCK OF G STREET NE  399886.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 491    136899.0  ...        5900.0    Precinct 143  38.899942   
         10824  135255.0  ...        6500.0     Precinct 89  38.885133   
         14887  137478.0  ...       10603.0    Precinct 144  38.905159   
         16277  135959.0  ...        8200.0     Precinct 89  38.891475   
         16280  136784.0  ...        8301.0     Precinct 83  38.898907   

                LONGITUDE           BID              START_DATE  \
20165648 491   -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         10824 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   
         14887 -77.005891          NOMA  2020/11/20 20:15:26+00   
         16277 -76.999516           NaN  2020/11/20 03:02:27+00   
         16280 -77.001314           NaN  2020/11/20 15:30:02+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 491    2020/11/20 22:08:28+00  499865401             NaN   
         10824                     NaN  500146815             NaN   
         14887  2020/11/20 21:46:24+00  500163163             NaN   
         16277                     NaN  500182525             NaN   
         16280  2020/11/20 18:25:35+00  500182528             NaN   

                               report_dt  
20165648 491   2020-11-20 22:07:10+00:00  
         10824 2020-11-20 12:46:32+00:00  
         14887 2020-11-20 22:17:27+00:00  
         16277 2020-11-20 04:27:36+00:00  
         16280 2020-11-20 18:56:18+00:00  

[5 rows x 26 columns]

In [22]:
df1 = df[df.WARD == 1].copy()
df1
#.copy() ensures that if you change anything in df it won't be reflected in df1

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
0,-77.033293,38.919188,20118678,2020/08/20 00:24:28+00,EVENING,OTHERS,THEFT/OTHER,1400 - 1499 BLOCK OF W STREET NW,397113.0,139035.0,...,4300.0,Precinct 22,38.919180,-77.033291,NaN,2020/08/08 00:24:44+00,2020/08/20 00:24:46+00,596090405,NaN,2020-08-20 00:24:28+00:00
7,-77.038004,38.930348,20120375,2020/08/23 14:20:13+00,DAY,OTHERS,THEFT/OTHER,3140 - 3172 BLOCK OF MOUNT PLEASANT STREET NW,396705.0,140274.0,...,2702.0,Precinct 39,38.930340,-77.038002,NaN,2020/08/23 13:50:59+00,2020/08/23 13:55:21+00,596090467,NaN,2020-08-23 14:20:13+00:00
12,-77.041588,38.925816,20401011,2020/07/21 18:40:02+00,DAY,OTHERS,THEFT/OTHER,2800 - 2899 BLOCK OF 18TH STREET NW,396394.0,139771.0,...,3901.0,Precinct 35,38.925808,-77.041586,NaN,2020/07/19 18:30:00+00,2020/07/20 00:00:00+00,596090485,NaN,2020-07-21 18:40:02+00:00
13,-77.046310,38.915409,20061534,2020/04/21 10:40:16+00,MIDNIGHT,OTHERS,BURGLARY,1811 - 1899 BLOCK OF CONNECTICUT AVENUE NW,395984.0,138616.0,...,4002.0,Precinct 25,38.915401,-77.046307,DUPONT CIRCLE,2020/04/21 09:10:18+00,2020/04/21 09:20:39+00,596090495,NaN,2020-04-21 10:40:16+00:00
16,-77.042956,38.917825,20401190,2020/08/02 00:09:54+00,EVENING,OTHERS,THEFT/OTHER,1800 - 1899 BLOCK OF CALIFORNIA STREET NW,396275.0,138884.0,...,4002.0,Precinct 25,38.917817,-77.042953,NaN,2020/08/01 22:30:00+00,2020/08/01 23:55:00+00,596090525,NaN,2020-08-02 00:09:54+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27904,-77.025199,38.920208,20080719,2020/06/01 04:46:56+00,MIDNIGHT,OTHERS,THEFT F/AUTO,2200 - 2299 BLOCK OF SHERMAN AVENUE NW,397815.0,139148.0,...,3500.0,Precinct 37,38.920200,-77.025196,NaN,2020/05/31 17:30:46+00,2020/06/01 02:50:05+00,596642611,NaN,2020-06-01 04:46:56+00:00
27910,-77.024218,38.918073,20178504,2020/12/17 08:41:49+00,MIDNIGHT,OTHERS,THEFT/OTHER,900 - 913 BLOCK OF FLORIDA AVENUE NW,397900.0,138911.0,...,4401.0,Precinct 137,38.918066,-77.024215,NaN,2020/12/17 08:00:36+00,2020/12/17 08:05:01+00,596642638,NaN,2020-12-17 08:41:49+00:00
27917,-77.040091,38.929690,20029123,2020/02/17 19:05:10+00,DAY,OTHERS,THEFT F/AUTO,1700 - 1799 BLOCK OF KENYON STREET NW,396524.0,140201.0,...,2702.0,Precinct 39,38.929682,-77.040089,NaN,2020/02/17 17:52:20+00,2020/02/17 17:53:26+00,596642688,NaN,2020-02-17 19:05:10+00:00
27920,-77.046026,38.923238,20174930,2020/12/09 21:38:05+00,EVENING,OTHERS,BURGLARY,1847 - 1999 BLOCK OF CALVERT STREET NW,396009.0,139485.0,...,4001.0,Precinct 25,38.923230,-77.046024,NaN,2020/12/04 19:35:02+00,2020/12/09 19:45:40+00,596642715,NaN,2020-12-09 21:38:05+00:00


## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [53]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    same_ward=C_Oth[ C_Oth['WARD']==r['WARD'] ]
    same_ward_same_time=same_ward[ (same_ward['report_dt']>=r['report_dt']) &
                                    (same_ward['report_dt']<=(r['report_dt'] + timedelta(minutes=1200)))].copy()
    #the previous same_ward_same_time is not a new dataframe, but a slice/view of the original dataframe. 

    store_matches_2[str(r['CCN'])]=same_ward_same_time

    return store_matches_2

### 2.2 apply it to one of the focal crimes

In [49]:
from datetime import datetime, timedelta


r = C_Tar.iloc[0]
find_related_crimes(r)
store_matches_2

{'20165648':                X          Y       CCN              REPORT_DAT     SHIFT  \
 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   EVENING   
 1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00  MIDNIGHT   
 4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00       DAY   
 11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   EVENING   
 15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00       DAY   
 15617 -76.997316  38.904969  20165803  2020/11/20 14:45:06+00       DAY   
 16271 -76.994365  38.900203  20165859  2020/11/20 15:37:59+00       DAY   
 25170 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00       DAY   
 
        METHOD              OFFENSE  \
 279    OTHERS  MOTOR VEHICLE THEFT   
 1308   OTHERS  MOTOR VEHICLE THEFT   
 4675   OTHERS         THEFT F/AUTO   
 11621  OTHERS          THEFT/OTHER   
 15616  OTHERS          THEFT/OTHER   
 15617  OTHERS         THEFT F/AUTO   
 16271  OTHERS          THEFT

### 2.3 Use apply to cover all the other focal crimes

In [55]:
C_Tar.apply(find_related_crimes, axis=1) #need to define the axis

941      {'20165648': ['X', 'Y', 'CCN', 'REPORT_DAT', '...
14013    {'20165648': ['X', 'Y', 'CCN', 'REPORT_DAT', '...
dtype: object